# Assignment 7

*(Due Saturday, November 2, 2019 at noon.)*

In [30]:
import numpy as np
import pandas as pd

1) Load either your dataset, `brfss`, or `nhis` and create at least three derived fields from the data there. Ideally, these will be fields you'll need for your final project.

In [31]:
#Setup for examples.
from urllib.request import urlretrieve
import zipfile
from pathlib import Path

zf = '../data/brfss/LLCP2018ASC.zip'
if not Path(zf).exists():
    Path('../data/brfss').mkdir(exist_ok=True)

    urlretrieve('https://www.cdc.gov/brfss/annual_data/2018/pdf/overview-2018-508.pdf', '../data/brfss/overview-2018-508.pdf')
    urlretrieve('https://www.cdc.gov/brfss/annual_data/2018/pdf/codebook18_llcp-v2-508.pdf', '../data/brfss/codebook18_llcp-v2-508.pdf')
    
    urlretrieve('https://www.cdc.gov/brfss/annual_data/2018/files/LLCP2018ASC.zip', zf)

fwff = '../data/brfss/LLCP2018.ASC'
if not Path(fwff).exists():
    with zipfile.ZipFile(zf) as z:
        z.extractall('../data/brfss')

In [32]:
names= ['state', 'imonth', 'iday', 
        'iyear', 'dispcode','genhlth', 
        'physhlth',
        'menthlth', 'poorhlth', 'hlthpln1',
        'persdoc2', 'medcost', 'checkup1',
        'WEIGHT2', 'HEIGHT3']
cols = [
    (1, 3),
    (18, 20),
    (20, 22),
    (22, 27),
    (31, 35),
    (89, 90),
    (90, 92),
    (92, 94),
    (94, 96),
    (96, 97),
    (97, 98),
    (98, 99),
    (99, 100),
    (176, 180),
    (180, 184)
]
types= {
    'WEIGHT2': str, 
    'HEIGHT3': str,
}
brfss = pd.read_fwf(fwff, 
                    names=names,
                    colspecs=cols,
                    dtype=types)

In [33]:
#I don't have cells that I need to derive from my data
#deriving cm from feet and inches
#must convert ft/in to just inches first
#divide height into 3 columns: units, inches, and feet
brfss.dropna(subset=['HEIGHT3'], inplace=True) 
#creating a height unit and height column from split HEIGHT3 column
brfss['htunit']=brfss.HEIGHT3.str[0].astype(np.int16)
brfss['ht']=brfss.HEIGHT3.str[1:]
#assign inches and feet
feet=brfss.ht.str[0].astype(np.uint8)
inches=brfss.ht.str[1:].astype(np.uint8)
#get total feet in inches and create htinches column
inches=inches+(feet*12)
brfss['htinches']=np.where(brfss.htunit==0, inches, brfss.ht)
brfss[brfss.htunit==9]
#get centimeters from inches 
cm=inches*0.4
brfss['htcm']=np.where(brfss.htunit==0, cm, brfss.ht)
brfss.head()

,state,imonth,iday,iyear,dispcode,genhlth,physhlth,menthlth,poorhlth,hlthpln1,persdoc2,medcost,checkup1,WEIGHT2,HEIGHT3,htunit,ht,htinches,htcm
0,1,1,5,2018,1100,2.0,30.0,88.0,30.0,1.0,1.0,2.0,1.0,0130,0504,0,504,64,25.6
1,1,1,12,2018,1100,3.0,88.0,88.0,NaN,2.0,1.0,1.0,2.0,0200,0505,0,505,65,26
2,1,1,8,2018,1100,5.0,10.0,88.0,88.0,1.0,1.0,2.0,1.0,0142,0410,0,410,58,23.2
3,1,1,3,2018,1100,1.0,88.0,88.0,NaN,1.0,1.0,2.0,1.0,0190,0510,0,510,70,28
4,1,1,12,2018,1100,2.0,88.0,88.0,NaN,1.0,2.0,2.0,1.0,7777,0503,0,503,63,25.2


In [34]:
#deriving kg from pounds
#divide weight into 2 columns: units and lbs
brfss.dropna(subset=['WEIGHT2'], inplace=True) 
#creating a weight unit and weight column from split WEIGHT2 column
brfss['wtunit']=brfss.WEIGHT2.str[0].astype(np.int16)
brfss['wt']=brfss.WEIGHT2.str[1:]
#assign kg
pounds=brfss.wt.str[0].astype(np.uint8)
kg=2.2*pounds
#create kg column
brfss['wtkg']=np.where(brfss.wtunit==0, kg, brfss.wt)
brfss[brfss.wtunit==9]
brfss.head()

,state,imonth,iday,iyear,dispcode,genhlth,physhlth,menthlth,poorhlth,hlthpln1,...,checkup1,WEIGHT2,HEIGHT3,htunit,ht,htinches,htcm,wtunit,wt,wtkg
0,1,1,5,2018,1100,2.0,30.0,88.0,30.0,1.0,...,1.0,0130,0504,0,504,64,25.6,0,130,2.2
1,1,1,12,2018,1100,3.0,88.0,88.0,NaN,2.0,...,2.0,0200,0505,0,505,65,26,0,200,4.4
2,1,1,8,2018,1100,5.0,10.0,88.0,88.0,1.0,...,1.0,0142,0410,0,410,58,23.2,0,142,2.2
3,1,1,3,2018,1100,1.0,88.0,88.0,NaN,1.0,...,1.0,0190,0510,0,510,70,28,0,190,2.2
4,1,1,12,2018,1100,2.0,88.0,88.0,NaN,1.0,...,1.0,7777,0503,0,503,63,25.2,7,777,777


In [35]:
#creating a day of year from day month and years column
#need to create complete date column first
brfss['Date']=pd.to_datetime(brfss[['iyear', 'imonth', 'iday']].rename(columns={'iyear': 'year', 
                                                                                'imonth': 'month', 'iday': 'day'}))
#pulling day of year from date column
brfss['day_of_yr']=brfss['Date'].dt.dayofyear
brfss.head()

,state,imonth,iday,iyear,dispcode,genhlth,physhlth,menthlth,poorhlth,hlthpln1,...,HEIGHT3,htunit,ht,htinches,htcm,wtunit,wt,wtkg,Date,day_of_yr
0,1,1,5,2018,1100,2.0,30.0,88.0,30.0,1.0,...,0504,0,504,64,25.6,0,130,2.2,2018-01-05,5
1,1,1,12,2018,1100,3.0,88.0,88.0,NaN,2.0,...,0505,0,505,65,26,0,200,4.4,2018-01-12,12
2,1,1,8,2018,1100,5.0,10.0,88.0,88.0,1.0,...,0410,0,410,58,23.2,0,142,2.2,2018-01-08,8
3,1,1,3,2018,1100,1.0,88.0,88.0,NaN,1.0,...,0510,0,510,70,28,0,190,2.2,2018-01-03,3
4,1,1,12,2018,1100,2.0,88.0,88.0,NaN,1.0,...,0503,0,503,63,25.2,7,777,777,2018-01-12,12


2) Run the following cell.

In [36]:
import seaborn as sns
sns.__version__

'0.9.0'

#### Submitting

In the Terminal (Mac) or Git Bash (Windows):

`git commit -a -m 'Assignment 7 completed'`  
`git push`

### Reading

1. [_Python for Data Analysis_](https://uri-primo.hosted.exlibrisgroup.com/permalink/f/10nopmq/01URI_ALMA51209454630002396):
    * Chapter 9